In [2]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt

## Question 2 - Read Files

In [27]:
def readFiles(ID):
    file = "part" + str(ID)
    pasta = "dataset\part" + str(ID)

    dev = [0]*5

    for i in range(5):
        d = []
        f = file + "dev" + str(i+1) + ".csv"
        dir = os.path.join(pasta, f)
        dev[i] = np.genfromtxt(dir, delimiter=',')

    return np.concatenate(dev)

array([[ 1.0000e+00, -1.8650e+00,  9.3890e+00, ...,  1.0134e+00,
         5.0589e+02,  1.0000e+00],
       [ 1.0000e+00, -1.7963e+00,  9.3742e+00, ...,  1.0000e+00,
         5.2542e+02,  1.0000e+00],
       [ 1.0000e+00, -1.8696e+00,  9.3000e+00, ...,  1.0156e+00,
         5.4495e+02,  1.0000e+00],
       ...,
       [ 5.0000e+00, -4.9463e-01,  9.4428e+00, ...,  7.1739e-02,
         1.0380e+06,  1.0000e+00],
       [ 5.0000e+00, -4.7013e-01,  9.4553e+00, ...,  8.4783e-02,
         1.0381e+06,  1.0000e+00],
       [ 5.0000e+00, -4.9427e-01,  9.4431e+00, ...,  7.8261e-02,
         1.0381e+06,  1.0000e+00]])

In [31]:
#data = [readFiles(i) for i in range(15)]
#all_data = np.concatenate(data)

data = readFiles(0)

## Question 3.1 - Boxplot

In [1]:
def boxplot(dados, title):
    plt.figure()
    plt.title(title)
    plt.boxplot(data)

def modulo():
    acel = [[0]]*16
    giro = [[0]]*16
    mag = [[0]]*16
    

    for row in data:
        acel[int(row[11])-1] += [np.sqrt(row[1]**2 + row[2]**2 + row[3]**2)] 
        giro[int(row[11])-1] += [np.sqrt(row[4]**2 + row[5]**2 + row[6]**2)] 
        mag[int(row[11])-1] += [np.sqrt(row[7]**2 + row[8]**2 + row[9]**2)] 
    
    print(acel)
    
    boxplot(acel, "Boxplot Aceleração")
    boxplot(giro, "Boxplot Giroscópio")
    boxplot(giro, "Boxplot Magnetómetro")

modulo()



NameError: name 'data' is not defined